In [1]:
import base64
import based58
import sys
import json
import os
import time
from pathlib import Path

import solana.rpc.api
from solders.signature import Signature
from solana.exceptions import SolanaRpcException
from solders.rpc.responses import GetTransactionResp

from src.tx_decoder import TransactionDecoder


%load_ext autoreload
%autoreload 2
sys.path.append('..')


ppk = "KLend2g3cP87fffoy8q1mQqGKjrxjC8boSyAYavgmjD"
idl_path = "../kamino_idl.json"


def get_tx(tx_signature):
    try:
        transaction = solana_client.get_transaction(
            tx_signature.signature, 
#             Signature.from_string(
#                 "4if5hzfKbJSxbA13ReBca2sY6kbAias7FseHG1QdUCHqSjhG4eNbGoz96TbBSNmCGoSQfKWx5gMtDVgxNb1iUV2M"
#             ),
            'jsonParsed', 
            max_supported_transaction_version=0
        )
        return transaction
    except SolanaRpcException as e:
        print(f"SolanaRpcException while fetching {str(tx_signature.signature)}")
        time.sleep(0.5)
        return get_tx(tx_signature)

    
def tx_printout(tx_parsed):
    print('===========logs============')
    for log, value in tx_parsed['logs'].items():
        print(f"{log} \n")
        print(f'      Parsed: {value} \n')
    
    print('=========relevant instructions=========')
    for program_id, instruction_data in tx_parsed['instructons']:
        print(f"Program ID: {program_id} \n   {instruction_data}")
    
    print('============Account Balances==========')
    for i in tx_parsed['account_balances']:
        print(i)
        
    print('============Token Balances==========')
    for i in tx_parsed['token_balances']:
        print(f"{i} \n")       
     

    
def decode(sign):
    ppk = "KLend2g3cP87fffoy8q1mQqGKjrxjC8boSyAYavgmjD"

    tx_decoder = TransactionDecoder(
        Path('../kamino_idl.json'),
        solana.rpc.api.Pubkey.from_string(ppk)
    )

    transaction = solana_client.get_transaction(
        Signature.from_string(
            sign
        ), 
        'jsonParsed', 
        max_supported_transaction_version=0
    )

    instructions, logs, acc_balance, tokens = tx_decoder.decode_transaction(transaction.value.transaction)

    tx_parsed = {
        'instructions': instructions, 'logs': logs,
        'account_balances': acc_balance, 'token_balances': tokens
    }
    return  tx_parsed

## Parse transactions from oldest to newest
- limited to one signer/ one mf account.


In [2]:
solana_client = solana.rpc.api.Client(os.getenv("RPC_URL"))

tx_decoder = TransactionDecoder(
    idl_path,  # mf ild
    solana.rpc.api.Pubkey.from_string(ppk)  # margin fi
)

In [3]:
signer = solana.rpc.api.Pubkey.from_string("3VRc3v3w7GJYRrdwNnfpFeh9YqeBZUVeWwe9BJ2hrhhE")

In [4]:
signer_tx_sinatures = solana_client.get_signatures_for_address(
    signer,
    limit = 1000
)
print(len(signer_tx_sinatures.value))

259


In [6]:
# collect signatures related to marginfi account:
account = solana.rpc.api.Pubkey.from_string("3VRc3v3w7GJYRrdwNnfpFeh9YqeBZUVeWwe9BJ2hrhhE")
acc_tx_sinatures = solana_client.get_signatures_for_address(
    account,
    limit = 1000
)
print(f"Transactions to collect: {len(acc_tx_sinatures.value)}")
# collect mf account related txs
acc_txs = list()

for idx, tx in enumerate(acc_tx_sinatures.value):
    print(idx+1)
    signature = tx.signature
    transaction = get_tx(tx)
    acc_txs.append(transaction)
print(f"Collected: {len(acc_txs)}")

Transactions to collect: 259
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
SolanaRpcException while fetching 2jPkAYnFKM6SUoBxH8Svyp1sUkmAxSHJ3hMteeHWpJVMKPLTX2Mqgyp4vEpcbJVspkMuRNsMG2NUvMkiPjHqVhoc
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
SolanaRpcException while fetching 3dbeFLRiysAzENL2Ci38tj3yEpw2yUnJj4kyuBU9NtQpDQemPZAng5xMcMDxpY3kWK5HMSY6osr7qH8Ajctxmii4
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
SolanaRpcException while fetching 5UhdA9XLdeW5NgL1Lw1Qx6K5nggpzxdsbZTM68qXHx8qN5Bd9cU84kh7iCksxxW6mB5q1gRApheo3J5nRkmDdzsJ
134
135
136
137
138
139
140
141
142
143
144
145
SolanaRpcException while fetching 5iUBoogYyfaE92ue5ntxD5JsUtYNvyBkbuEZYh4hJSNrckHepeippo6ho9fvyEH915WwV5c6ygneke4C3keafHNk
146
147

In [8]:
def dump_json_to_file(json_data, file_path):
    with open(file_path, 'w') as file:
        json.dump(json_data, file, indent=4)

def load_json_from_file(file_path):
    with open(file_path, 'r') as file:
        return json.load(file)


# path = './mf_txs/'
# for num, tx in enumerate(mfacc_txs):
#     dump_json_to_file(tx.to_json(), os.path.join(path, f"{'{:03d}'.format(num)}.json"))

In [9]:
acc_txs.reverse()

## GET data ffrom local storage:

In [11]:

for i, tx in enumerate(acc_txs):
    print(f"============ {i} =============", str(tx.value.transaction.transaction.signatures[0]))
    for log in tx.value.transaction.meta.log_messages:
        print(log)

============ 0 ============= 4UKQratjFEMEBdFSu4uTBRcNWfBKfp6Y2QgkmXnnkanHzGZMKpinkKPyEkSiYB85xBQnuJPGuXrk9K44WJRF7KTE
Program 11111111111111111111111111111111 invoke [1]
Program 11111111111111111111111111111111 success
Program ComputeBudget111111111111111111111111111111 invoke [1]
Program ComputeBudget111111111111111111111111111111 success
============ 1 ============= 48VB2HCQH13NNChSVHcQHh7wJUzNdDR6wQY62YsWrDFzVWsaXHcLoTWfbUy8fGhXL63Jp2wggG4bbjXMZf2DRS4o
Program 11111111111111111111111111111111 invoke [1]
Program 11111111111111111111111111111111 success
Program ComputeBudget111111111111111111111111111111 invoke [1]
Program ComputeBudget111111111111111111111111111111 success
============ 2 ============= 38da5gih9aY7BeCq8jqUVEXAPBZqw4QeLDy1RGGfH6UcrGJvsd7AjdXLbyRdZDK2ZJxbUi74fUshNWYgQcLm75tX
Program ComputeBudget111111111111111111111111111111 invoke [1]
Program ComputeBudget111111111111111111111111111111 success
Program ComputeBudget111111111111111111111111111111 invoke [1]
Program Comp

In [7]:
# print(f"Events collected: {len(tx_decoder.events)}")

# for tx in mfacc_txs:
#     tx_decoder.decode(tx.value.transaction, tx.value.slot)
    
# print(f"Events collected: {len(tx_decoder.events)}") 

Events collected: 0
Events collected: 0


In [ ]:
instructions = ['initLendingMarket',
 'updateLendingMarket',
 'updateLendingMarketOwner',
 'initReserve',
 'initFarmsForReserve',
 'updateSingleReserveConfig',
 'updateEntireReserveConfig',
 'refreshReserve',
 'depositReserveLiquidity',
 'redeemReserveCollateral',
 'initObligation',
 'initObligationFarmsForReserve',
 'refreshObligationFarmsForReserve',
 'refreshObligation',
 'depositObligationCollateral',
 'withdrawObligationCollateral',
 'borrowObligationLiquidity',
 'repayObligationLiquidity',
 'depositReserveLiquidityAndObligationCollateral',
 'withdrawObligationCollateralAndRedeemReserveCollateral',
 'liquidateObligationAndRedeemReserveCollateral',
 'redeemFees',
 'flashRepayReserveLiquidity',
 'flashBorrowReserveLiquidity',
 'socializeLoss',
 'requestElevationGroup',
 'initReferrerTokenState',
 'initUserMetadata',
 'withdrawReferrerFees',
 'withdrawProtocolFee',
 'initReferrerStateAndShortUrl',
 'deleteReferrerStateAndShortUrl',
 'updateUserMetadataOwner']

entity = dict()

for i, tx in enumerate(acc_txs):
    signature = str(acc_txs[0].value.transaction.transaction.signatures[0])
#     decoded_tx = decode(signature)
    logs = tx.value.transaction.meta.log_messages
    print(f"=================== {i} ================")
#     print(logs)
    for l in logs:
        if [instr for instr in instructions if instr.lower() in l.lower()]:
            print(l)
